<h1> UFO Sightings 🛸 </h1>
<h3> Exploratory Data Analysis (EDA) </h3>
<p>Data obtained from Kaggle: </p>
https://www.kaggle.com/datasets/ogunkoya/ufo-1149

Before starting, importing all the libraries needed

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta

<h4>Step 1: Exploring Data Available</h4>

In [2]:
# first, import the data into a pandas dataframe
df = pd.read_csv('UFO-1149.csv')

In [3]:
# first 3 records in the dataframe
df.head(3)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8831,-97.9411
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.3842,-98.5811
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2000,-2.9167


In [4]:
# number of rows and columns
df.shape

(80332, 11)

In [5]:
# reviewing all columns, data types, and amount of records in each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  float64
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80318 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  float64
 10  longitude             80332 non-null  float64
dtypes: float64(3), object(8)
memory usage: 6.7+ MB


In [6]:
# missing values in each column
df.isnull().sum()

datetime                   0
city                       0
state                   5797
country                 9670
shape                   1932
duration (seconds)         0
duration (hours/min)       0
comments                  14
date posted                0
latitude                   0
longitude                  0
dtype: int64

In [7]:
# unique items in each column
df.nunique()

datetime                69586
city                    19900
state                      67
country                     5
shape                      29
duration (seconds)        533
duration (hours/min)     8304
comments                79998
date posted               317
latitude                17834
longitude               19192
dtype: int64

In [8]:
# looking at all unique countries recorded
df['country'].value_counts()

us    65114
ca     3000
gb     1905
au      538
de      105
Name: country, dtype: int64

In [9]:
# looking at all the unique values under state column
# some non US states listed
df['state'].unique()

array(['tx', nan, 'hi', 'tn', 'ct', 'al', 'fl', 'ca', 'nc', 'ny', 'ky',
       'mi', 'ma', 'ks', 'sc', 'wa', 'ab', 'co', 'nh', 'wi', 'me', 'ga',
       'pa', 'il', 'ar', 'on', 'mo', 'oh', 'in', 'az', 'mn', 'nv', 'nf',
       'ne', 'or', 'bc', 'ia', 'va', 'id', 'nm', 'nj', 'mb', 'wv', 'ok',
       'ri', 'nb', 'vt', 'la', 'pr', 'ak', 'ms', 'ut', 'md', 'mt', 'sk',
       'wy', 'sd', 'pq', 'ns', 'qc', 'de', 'nd', 'dc', 'nt', 'sa', 'yt',
       'yk', 'pe'], dtype=object)

<hr>

<b>Quick Observations: </b>
- Need to reformat 'datetime' column to separate the time and date <br>
- Don't need 'date posted' or 'comments' columns <br>
- Slightly over 80,000 entries with some missing data in state, country, and shape <br>
- Need to clean up the US states and some foreign entries seemed to be lumped into the 'city' column <br>

<br>

<b>Questions I want to answer with this data: </b>
1. What time periods are captured in this dataset? <br>
2. Are there multiple reports from the same day? <br>
3. Do most sightings occur during the day or at night? <br>
4. What states have the most sightings? <br>
5. How long do sightings last on average? <br>

<hr>

<h4>Section 2: Data Cleaning and Transformation</h4>

In [10]:
#replace 24:00 hours with 0:00 in order to use datetime features
df['fix_time'] = df['datetime'].str.replace('24:00', '0:00')

#recasting data type to datetime
df['tempdatetime'] = pd.to_datetime(df['fix_time'])

#items that contain 24:00 hour only
rows = df['datetime'].str.contains('24:00')

#applying update to fix the hour and save in a new column
df['datetime_fixed'] = df['tempdatetime'] + rows * timedelta(days=1)

In [11]:
# splitting datetime column to new 'date' column
df['date'] = pd.to_datetime(df['datetime_fixed']).dt.date

In [12]:
# creating a new column from the split above for 'time'
df['time'] = pd.to_datetime(df['datetime_fixed']).dt.time

In [13]:
# recasting datatype
df['date posted'] = pd.to_datetime(df['date posted']).dt.date

In [14]:
# creating a new column that shows which day of the week the sighting occurred
df['day'] = df['datetime_fixed'].dt.day_name()

In [15]:
# setting list of all us states
states = [ 'ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
           'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
           'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
           'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
           'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

In [16]:
# adding country as 'us' on items where the country is NULL and the state is listed in variable above
c1 = df['country'].isna()
c2 = df['state'].isin(states)

df['country'] = df['country'].mask(c1 & c2, 'us')

In [17]:
# redo 'duration (seconds)' to a new more meaningful column 'duration'
def time(x):
    min, sec = divmod(x, 60)
    hour, min = divmod(min, 60)
    new_time =  '%d:%02d:%02d' % (hour, min, sec)  #saves time in HH:MM:SS format
    return new_time

df['duration'] = df['duration (seconds)'].apply(time)

In [18]:
# dropping columns that won't be needed
df.drop(['datetime', 'duration (hours/min)', 'fix_time', 'tempdatetime'], axis = 1, inplace=True)

In [19]:
# removing rows with missing data in the following columns
df.dropna(how='any', axis = 0, subset=['country', 'shape', 'state'], inplace=True)

In [20]:
# no more missing data
df.isnull().sum()

city                  0
state                 0
country               0
shape                 0
duration (seconds)    0
comments              7
date posted           0
latitude              0
longitude             0
datetime_fixed        0
date                  0
time                  0
day                   0
duration              0
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72152 entries, 0 to 80331
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   city                72152 non-null  object        
 1   state               72152 non-null  object        
 2   country             72152 non-null  object        
 3   shape               72152 non-null  object        
 4   duration (seconds)  72152 non-null  float64       
 5   comments            72145 non-null  object        
 6   date posted         72152 non-null  object        
 7   latitude            72152 non-null  float64       
 8   longitude           72152 non-null  float64       
 9   datetime_fixed      72152 non-null  datetime64[ns]
 10  date                72152 non-null  object        
 11  time                72152 non-null  object        
 12  day                 72152 non-null  object        
 13  duration            72152 non-null  object    

In [22]:
# confirming updates
df.head()

,city,state,country,shape,duration (seconds),comments,date posted,latitude,longitude,datetime_fixed,date,time,day,duration
0,san marcos,tx,us,cylinder,2700.0,This event took place in early fall around 194...,2004-04-27,29.8831,-97.9411,1949-10-10 20:30:00,1949-10-10,20:30:00,Monday,0:45:00
1,lackland afb,tx,us,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.3842,-98.5811,1949-10-10 21:00:00,1949-10-10,21:00:00,Monday,2:00:00
3,edna,tx,us,circle,20.0,My older brother and twin sister were leaving ...,2004-01-17,28.9783,-96.6458,1956-10-10 21:00:00,1956-10-10,21:00:00,Wednesday,0:00:20
4,kaneohe,hi,us,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4181,-157.8036,1960-10-10 20:00:00,1960-10-10,20:00:00,Monday,0:15:00
5,bristol,tn,us,sphere,300.0,My father is now 89 my brother 52 the girl wit...,2007-04-27,36.5950,-82.1889,1961-10-10 19:00:00,1961-10-10,19:00:00,Tuesday,0:05:00


In [23]:
# saving dataframe to new csv file
#df.to_csv('clean_ufo_data.csv', index=False)

<h4>Section 3: Analysis & Visualization</h4>

<p>1. What time periods are captured in this dataset?</p>

In [24]:
df['date'].agg(['min', 'max'])

min    1910-01-02
max    2014-05-08
Name: date, dtype: object

In [25]:
df['date posted'].agg(['min', 'max'])

min    1998-03-07
max    2014-05-08
Name: date posted, dtype: object

<p>This dataset has sightings from early 1910 through 2014. The date of occurrence doesn’t always align with the data posted. The earliest posted record was from 1998. </br>

<p>2. Are there multiple reports from the same day?


In [26]:
print("Number of unique date variations: ",df['date'].nunique())

Number of unique date variations:  10084


In [27]:
#looking at first 5 dates with the most sightings
df['date'].value_counts()[:5]

2010-07-04    199
2012-07-04    182
1999-11-16    171
2013-07-04    170
2011-07-04    142
Name: date, dtype: int64

<p>July 4th seems to be a hot day for UFO sightings. Wonder if that has to do with fireworks in the US for Independence Day. Something to look more into, possibly see which countries these sights are from.

<p>3. Do most sightings occur during the day or at night?

To answer this question the time needs to be separated into a day/night category. Day occurrences will be between 06:01 - 18:00, night 18:01 - 06:00.

In [28]:
# first to find items between that time, then relabel to day/night
df['day_night'] = df['datetime_fixed'].dt.hour.between(6, 18, inclusive=True)
df['day_night'] = df['day_night'].replace({True: "Day", False: "Night"})

/var/folders/d4/87jlcn2n55q6fk4n266w8wjc0000gn/T/ipykernel_83014/3649366154.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df['day_night'] = df['datetime_fixed'].dt.hour.between(6, 18, inclusive=True)


In [29]:
df['day_night'].value_counts()

Night    54252
Day      17900
Name: day_night, dtype: int64

<p>4. What states have the most sightings?

<p>5. How long do sightings last on average?